# Mistral and Weights & Biases

- Weights & Biases: https://wandb.ai/
- Mistral finetuning docs: https://docs.mistral.ai/capabilities/finetuning/
- Tracing with W&B Weave: https://wandb.me/weave

In [1]:
# !pip install mistralai pandas weave

## Using Mistral and Weave

You will probably integrate MistralAI API calls in your codebase by creating a function like the one below:

In [5]:
import os
import weave
from mistralai.client import MistralClient
from mistralai.models.chat_completion import ChatMessage

client = MistralClient(api_key=os.environ["MISTRAL_API_KEY"])

@weave.op()  # <---- add this and you are good to go
def call_mistral(model:str, messages:list) -> str:
    "Call the Mistral API"
    chat_response = client.chat(
        model=model,
        messages=messages,
    )
    return chat_response.choices[0].message.content

The only thing you need to do is add the @weave.op() decorator to the function you want to trace.

Let's define a more interesting function that recommends cheese based on the region and model.



In [ ]:
@weave.op()
def cheese_recommender(region:str, model:str) -> str:
    "Recommend the best cheese in a given region"
     
    messages = [ChatMessage(
        role="user", 
        content=f"What is the best cheese in {region}?")]

    cheeses = call_mistral(model=model, messages=messages)
    return {"region": region, "cheeses": cheeses}

Let's run this function and see how weave traces it. We call weave.init() to tell weave the project where to store the traces.

In [ ]:

weave.init("mistral_webinar")
print(cheese_recommender(region="France", model="open-mistral-7b"))

You can view the traces by clicking the link above 👆
![](cheese_recomender.png)



## Prepare the dataset

Some data from wandbot

In [14]:
import pandas as pd
df = pd.read_json('qa.jsonl', orient='records', lines=True)

In [25]:
df.head()

,question,answer
0,What is the difference between team and organi...,A team is a collaborative workspace for a grou...
1,What is the difference between team and entity...,A team is a collaborative workspace for a grou...
2,What is a team and where can I find more infor...,Use W&B Teams as a central workspace for your ...
3,When should I log to my personal entity agains...,You should log to your personal entity when yo...
4,Who can create a team? Who can add or delete p...,**Admin**: Team admins can add and remove othe...


A neat trick to get better answers is instead of passing a very long initial message, passing a small conversation with some prefilled agent responses.

In [23]:
@weave.op()
def wandb_expert(question:str, model:str) -> str:
    "Answer questions about wandb"
     
    messages = [
        ChatMessage(
            role="user", 
            content=(
                "You are an expert about Weights & Biases the ML platform. "
                 "You will answer questions about the product, Answer the question directly, without repeating the instructions."
                 )
        ),
        ChatMessage(
            role="assistant", 
            content=(
                "Sure, I'd be happy to help with your question about Weights & Biases. "
                 "If you have a specific question about using Weights & Biases, such as how to track experiments, "
                 "visualize data, or manage artifacts, please feel free to ask!")
        ),
        ChatMessage(
            role="user", 
            content=f"Here is the question: {question}"
        )
    ]

    answer = call_mistral(model=model, messages=messages)
    return {"question": question, "answer": answer}

res = wandb_expert(question=df.loc[0].question, model="open-mistral-7b")
print(df.loc[0].question)
print(res["answer"])

🍩 https://wandb.ai/capecape/mistral_webinar/r/call/78c127e1-6128-4600-867c-4e6a575ea441
What is the difference between team and organization?
In Weights & Biases (W&B), a Team represents a group of users who collaborate within a project. A Team can have multiple Members, and Members can create and manage Experiments and Projects within the Team.

An Organization, on the other hand, is a higher-level entity that groups Teams together. An Organization can have multiple Teams and is typically used to manage access and billing for multiple projects or departments within a company.

So, in summary, a Team is a collaborative unit within a project, while an Organization is a higher-level entity that groups Teams together and manages access and billing.


This is a very random answer, without following the instruction nor knowing about the question itself. Let's try `mistral-large` for comparison.

In [24]:
res = wandb_expert(question=df.loc[0].question, model="mistral-large-latest")
print(df.loc[0].question)
print(res["answer"])

🍩 https://wandb.ai/capecape/mistral_webinar/r/call/7e529e77-6a4c-43da-96de-f631ed752a70
What is the difference between team and organization?
In Weights & Biases, a team is a group of users who share access to a set of experiments and other resources, such as models and datasets. Teams are typically used to collaborate on a specific project or set of projects.

An organization, on the other hand, is a higher-level entity that can contain multiple teams. Organizations are typically used to represent companies or other larger groups, and they provide a way to manage access to resources and enforce policies across multiple teams. For example, an organization might have multiple teams working on different projects, each with their own set of experiments and resources.

One key difference between teams and organizations is that organizations have additional features, such as the ability to create and manage custom roles and permissions, and to set up single sign-on (SSO) for authentication.

This is pretty descent for both 😍. Let's see if fine-tuning improves this.

In [42]:
def create_messages(row):
    "Format on the expected MistralAI fine-tuning dataset"
    question = row['question']
    answer = row['answer']
    msgs = dict(messages=[
        dict(
            role="user", 
            content=(
                "You are an expert about Weights & Biases the ML platform. "
                "You will answer questions about the product, Answer the question directly, without repeating the instructions."
            )
        ),
        dict(
            role="assistant", 
            content=(
                "Sure, I'd be happy to help with your question about Weights & Biases. "
                "If you have a specific question about using Weights & Biases, such as how to track experiments, "
                "visualize data, or manage artifacts, please feel free to ask!"
            )
        ),
        dict(
            role="user", 
            content=f"Here is the question: {question}"
        ),
        dict(
            role="assistant", 
            content=answer
        )
    ])
    return msgs

# To use this function on a pandas DataFrame:
# df['messages'] = df.apply(create_messages, axis=1)



In [46]:
msgs = create_messages(df.loc[0])
msgs

{'messages': [{'role': 'user',
   'content': 'You are an expert about Weights & Biases the ML platform. You will answer questions about the product, Answer the question directly, without repeating the instructions.'},
  {'role': 'assistant',
   'content': "Sure, I'd be happy to help with your question about Weights & Biases. If you have a specific question about using Weights & Biases, such as how to track experiments, visualize data, or manage artifacts, please feel free to ask!"},
  {'role': 'user',
   'content': 'Here is the question: What is the difference between team and organization?'},
  {'role': 'assistant',
   'content': 'A team is a collaborative workspace for a group of users working on the same projects, while an organization is a higher-level entity that may consist of multiple teams and is often related to billing and account management.'}]}

In [48]:
df = df.apply(create_messages, axis=1)
df.head()

0    {'messages': [{'role': 'user', 'content': 'You...
1    {'messages': [{'role': 'user', 'content': 'You...
2    {'messages': [{'role': 'user', 'content': 'You...
3    {'messages': [{'role': 'user', 'content': 'You...
4    {'messages': [{'role': 'user', 'content': 'You...
dtype: object

In [56]:
df_train=df.sample(frac=0.9,random_state=200)
df_eval=df.drop(df_train.index)

df_train.to_json("train.jsonl", orient="records", lines=True)
df_eval.to_json("eval.jsonl", orient="records", lines=True)

## Upload dataset

In [57]:
import os
from mistralai.client import MistralClient

api_key = os.environ.get("MISTRAL_API_KEY")
client = MistralClient(api_key=api_key)

with open("train.jsonl", "rb") as f:
    ds_train = client.files.create(file=("train.jsonl", f))
with open("eval.jsonl", "rb") as f:
    ds_eval = client.files.create(file=("eval.jsonl", f))


In [58]:
import json
def pprint(obj):
    print(json.dumps(obj.dict(), indent=4))

In [59]:
pprint(ds_train)

{
    "id": "78c99fec-4b0c-4a3f-b8f9-5473cf533753",
    "object": "file",
    "bytes": 147176,
    "created_at": 1719332567,
    "filename": "train.jsonl",
    "purpose": "fine-tune"
}


In [60]:
pprint(ds_eval)

{
    "id": "57c1d237-8c88-4b0c-8b5a-c7c7be283761",
    "object": "file",
    "bytes": 15339,
    "created_at": 1719332568,
    "filename": "eval.jsonl",
    "purpose": "fine-tune"
}


## Create a fine-tuning job

In [71]:
from mistralai.models.jobs import TrainingParameters, WandbIntegrationIn

created_jobs = client.jobs.create(
    model="open-mistral-7b",
    training_files=[ds_train.id],
    validation_files=[ds_eval.id],
    hyperparameters=TrainingParameters(
        training_steps=10,
        learning_rate=0.0001,
        ),
    integrations=[
        WandbIntegrationIn(
            project="mistral_webinar",
            run_name="finetune_wandb",
            api_key=os.environ.get("WANDB_API_KEY"),
        ).dict()
    ],
)

In [72]:
pprint(created_jobs)

{
    "id": "64861646-2420-439a-97b0-739f35ca84bc",
    "hyperparameters": {
        "training_steps": 10,
        "learning_rate": 0.0001
    },
    "fine_tuned_model": null,
    "model": "open-mistral-7b",
    "status": "QUEUED",
    "job_type": "FT",
    "created_at": 1719342089,
    "modified_at": 1719342089,
    "training_files": [
        "78c99fec-4b0c-4a3f-b8f9-5473cf533753"
    ],
    "validation_files": [
        "57c1d237-8c88-4b0c-8b5a-c7c7be283761"
    ],
    "object": "job",
    "integrations": [
        {
            "type": "wandb",
            "project": "mistral_webinar",
            "name": null,
            "run_name": "finetune_wandb"
        }
    ]
}


In [73]:
import time

retrieved_job = client.jobs.retrieve(created_jobs.id)
while retrieved_job.status in ["RUNNING", "QUEUED"]:
    retrieved_job = client.jobs.retrieve(created_jobs.id)
    pprint(retrieved_job)
    print(f"Job is {retrieved_job.status}, waiting 10 seconds")
    time.sleep(10)



{
    "id": "64861646-2420-439a-97b0-739f35ca84bc",
    "hyperparameters": {
        "training_steps": 10,
        "learning_rate": 0.0001
    },
    "fine_tuned_model": null,
    "model": "open-mistral-7b",
    "status": "RUNNING",
    "job_type": "FT",
    "created_at": 1719342089,
    "modified_at": 1719342089,
    "training_files": [
        "78c99fec-4b0c-4a3f-b8f9-5473cf533753"
    ],
    "validation_files": [
        "57c1d237-8c88-4b0c-8b5a-c7c7be283761"
    ],
    "object": "job",
    "integrations": [
        {
            "type": "wandb",
            "project": "mistral_webinar",
            "name": null,
            "run_name": "finetune_wandb"
        }
    ],
    "events": [
        {
            "name": "status-updated",
            "data": {
                "status": "RUNNING"
            },
            "created_at": 1719342089
        },
        {
            "name": "status-updated",
            "data": {
                "status": "QUEUED"
            },
            "

In [74]:
# List jobs
jobs = client.jobs.list()
pprint(jobs)

{
    "data": [
        {
            "id": "64861646-2420-439a-97b0-739f35ca84bc",
            "hyperparameters": {
                "training_steps": 10,
                "learning_rate": 0.0001
            },
            "fine_tuned_model": "ft:open-mistral-7b:0362203c:20240625:64861646",
            "model": "open-mistral-7b",
            "status": "SUCCESS",
            "job_type": "FT",
            "created_at": 1719342089,
            "modified_at": 1719342173,
            "training_files": [
                "78c99fec-4b0c-4a3f-b8f9-5473cf533753"
            ],
            "validation_files": [
                "57c1d237-8c88-4b0c-8b5a-c7c7be283761"
            ],
            "object": "job",
            "integrations": [
                {
                    "type": "wandb",
                    "project": "mistral_webinar",
                    "name": null,
                    "run_name": "finetune_wandb"
                }
            ]
        },
        {
            "id": "8e24

In [75]:
# Retrieve a jobs
retrieved_jobs = client.jobs.retrieve(created_jobs.id)
pprint(retrieved_jobs)


{
    "id": "64861646-2420-439a-97b0-739f35ca84bc",
    "hyperparameters": {
        "training_steps": 10,
        "learning_rate": 0.0001
    },
    "fine_tuned_model": "ft:open-mistral-7b:0362203c:20240625:64861646",
    "model": "open-mistral-7b",
    "status": "SUCCESS",
    "job_type": "FT",
    "created_at": 1719342089,
    "modified_at": 1719342173,
    "training_files": [
        "78c99fec-4b0c-4a3f-b8f9-5473cf533753"
    ],
    "validation_files": [
        "57c1d237-8c88-4b0c-8b5a-c7c7be283761"
    ],
    "object": "job",
    "integrations": [
        {
            "type": "wandb",
            "project": "mistral_webinar",
            "name": null,
            "run_name": "finetune_wandb"
        }
    ],
    "events": [
        {
            "name": "status-updated",
            "data": {
                "status": "SUCCESS"
            },
            "created_at": 1719342173
        },
        {
            "name": "status-updated",
            "data": {
                "sta

## Use a fine-tuned model

In [87]:
df_eval.iloc[1]

{'messages': [{'role': 'user',
   'content': 'You are an expert about Weights & Biases the ML platform. You will answer questions about the product, Answer the question directly, without repeating the instructions.'},
  {'role': 'assistant',
   'content': "Sure, I'd be happy to help with your question about Weights & Biases. If you have a specific question about using Weights & Biases, such as how to track experiments, visualize data, or manage artifacts, please feel free to ask!"},
  {'role': 'user',
   'content': 'Here is the question: What is the difference between `.log()` and `.summary`?'},
  {'role': 'assistant',
   'content': 'The summary is the value that shows in the table while the log will save all the values for plotting later.\n\nFor example, you might want to call `wandb.log` every time the accuracy changes. Usually, you can just use .log. `wandb.log()` will also update the summary value by default unless you have set the summary manually for that metric\n\nThe scatterplo

In [88]:
wandb_expert(question="What is the difference between `.log()` and `.summary`?", 
             model=retrieved_jobs.fine_tuned_model)

🍩 https://wandb.ai/capecape/mistral_webinar/r/call/ee105e61-0b1d-4699-94d1-cac947436cb1


{'question': 'What is the difference between `.log()` and `.summary`?',
 'answer': 'The `.log()` method writes a single line to the log. The `. summary` method writes the tensor data as a table in the log. The log is a stream of text, so only the last 500 lines are shown by default in the UI. For example:'}